In [9]:
pip install internetarchive

Note: you may need to restart the kernel to use updated packages.


In [1]:
from internetarchive import download
from internetarchive import get_item
import requests
import os
import pandas as pd
from io import BytesIO
from PIL import Image

In [2]:
# query path

search = 'map'
subject = 'data visualization'

params = {
    'q': f'({search}) AND subject:("{subject}") AND mediatype:(image)',
    'fl[]': 'identifier',
    'rows': 100,
    # 'sort[]': 'views desc',
    'sort[]': 'downloads desc', 
    'output': 'json'
}

search_url = 'https://archive.org/advancedsearch.php'
response = requests.get(search_url, params=params)
results = response.json()['response']['docs']

identifiers = [doc.get('identifier') for doc in results]
print(f"Found {len(identifiers)} items.")

identifiers

# 'fl[]': 'title',
# 'fl[]': 'subject',
# 'fl[]': 'date',
# 'fl[]': 'description',

Found 100 items.


['dr_composite-map-of-tavola-1-60-map-of-the-world-with-additional-spheres-10130087',
 'dr_composite-map-mappemonde-dress-es-en-1450-par-fra-mauro-cosmographe-vene-11177007',
 'dr_page-4-adams-synchronological-chart-of-universal-history-through-the-ey-6000005',
 'dr_composite-of-adams-synchronological-chart-of-universal-history-through-6000013',
 'dr_the-histomap-of-evolution-earth-life-and-mankind-for-ten-thousand-million-9869002',
 'dr_the-histomap-four-thousand-years-of-world-history-relative-power-of-conte-1810001',
 'dr_cosmological-diagram-the-world-of-mortals-12047000',
 'dr_the-histomap-of-religion-the-story-of-mans-search-for-spiritual-unity-by-10260005',
 'dr_the-geologic-time-spiral-a-path-to-the-past-general-interest-publication-5-10627000',
 'dr_page-6-adams-synchronological-chart-of-universal-history-through-the-ey-6000007',
 'dr_page-1-adams-synchronological-chart-of-universal-history-through-the-ey-6000002',
 'dr_panoramic-plan-of-the-principal-rivers-and-lakes-drawn--e

In [ ]:
# loop to download the largest image files for each item in the search result

download_dir = '/Users/aliceviggiani/Dropbox/1 estudos/5 data viz/0 master/pratt/2.5 2025 summer/697-botz/2-assignments/3 maprobot/img_original'

urls = []

for i in identifiers:
    item = get_item(i)
    sizes = []
    names = []

    for f in item.files:
        if f['name'].lower().endswith(('.jpg', '.jpeg', '.png')):
            sizes.append(int(f['size']))
            names.append(f['name'])

    if sizes:
        index = sizes.index(max(sizes))   
        largest_file = item.get_file(names[index])
        largest_file.download(
            destdir='/Users/aliceviggiani/Dropbox/1 estudos/5 data viz/0 master/pratt/2.5 2025 summer/697-botz/2-assignments/3 maprobot/img_original')
        urls.append(largest_file.url)
    else:
        print("Error")

urls

['https://archive.org/download/dr_composite-map-of-tavola-1-60-map-of-the-world-with-additional-spheres-10130087/10130087.jpg',
 'https://archive.org/download/dr_composite-map-mappemonde-dress-es-en-1450-par-fra-mauro-cosmographe-vene-11177007/11177007.jpg',
 'https://archive.org/download/dr_page-4-adams-synchronological-chart-of-universal-history-through-the-ey-6000005/6000005.jpg',
 'https://archive.org/download/dr_composite-of-adams-synchronological-chart-of-universal-history-through-6000013/6000013.jpg',
 'https://archive.org/download/dr_the-histomap-of-evolution-earth-life-and-mankind-for-ten-thousand-million-9869002/9869002.jpg',
 'https://archive.org/download/dr_the-histomap-four-thousand-years-of-world-history-relative-power-of-conte-1810001/1810001.jpg',
 'https://archive.org/download/dr_cosmological-diagram-the-world-of-mortals-12047000/12047000.jpg',
 'https://archive.org/download/dr_the-histomap-of-religion-the-story-of-mans-search-for-spiritual-unity-by-10260005/10260005.j

In [4]:
# # croping image to a valid Instagram aspect ratio

# def crop_square(image, size=(600, 600)):
#     # original dimensions
#     width, height = image.size

#     # determine the size of the square
#     min_side = min(width, height)

#     # calculate crop box for center square
#     left = (width - min_side) // 2
#     top = (height - min_side) // 2
#     right = left + min_side
#     bottom = top + min_side

#     # crop to square and resize
#     image = image.crop((left, top, right, bottom))
#     image = image.resize(size, Image.ANTIALIAS)
#     return image


# for u in urls:
#     response = requests.get(u)
#     image_original = Image.open(BytesIO(response.content))

#     image_cropped = crop_square(image_original)

#     filename = os.path.basename(u)
#     save_path = os.path.join('/Users/aliceviggiani/Dropbox/1 estudos/5 data viz/0 master/pratt/2.5 2025 summer/697-botz/2-assignments/3 maprobot/img-sq', f'{filename}')
#     image_cropped.save(save_path)

In [7]:
# resizing image to fit Instagram aspect ratio

# accepted format on Instagram:
# width: 320 - 1080 pixels
# height: 566 – 1350
# aspect ratio 1.91:1 – 4:5 


def resize(image):
    # original dimensions
    width, height = image.size
    ratio = width / height

    # maximum format accepted by instagram
    width_max = 1080
    ratio_min = 0.8
    ratio_max = 1.91

    # resize if needed, while keeping the same ratio 
    if width > width_max or not(ratio_min <= ratio <= ratio_max):
        image.thumbnail((width_max, width_max), Image.LANCZOS)
        return image
    else:
        return image


for u in urls:
    response = requests.get(u)
    image_original = Image.open(BytesIO(response.content))

    image_resized = resize(image_original)

    filename = os.path.basename(u)
    save_path = os.path.join('/Users/aliceviggiani/Dropbox/1 estudos/5 data viz/0 master/pratt/2.5 2025 summer/697-botz/2-assignments/3 maprobot/img_resized', f'{filename}')
    image_resized.save(save_path)

In [8]:
# loop to get info for the legends

titles = []
dates = []
# descriptions = []
subjects = []
languages = []
collections = []

for i in identifiers:
    item = get_item(i)

    title = item.metadata.get('title', '[None]')
    date = item.metadata.get('date', '[None]')
    description = item.metadata.get('description', '[None]')
    subject = item.metadata.get('subject', '[None]')
    language = item.metadata.get('language', '[None]')
    collection = item.metadata.get('collection', '[None]')
    
    titles.append(title)
    dates.append(date)
    # descriptions.append(description)
    subjects.append(subject)
    languages.append(language)
    collections.append(collection)

In [ ]:
subjects

[['Data Visualization', 'Art'],
 'Data Visualization',
 ['Historical', 'Data Visualization'],
 ['Pictorial map', 'Historical', 'Data Visualization'],
 ['Pictorial map', 'Data Visualization'],
 ['Pictorial map', 'Historical', 'Data Visualization'],
 ['Religions', 'Pictorial map', 'Celestial', 'Data Visualization'],
 ['Pictorial map', 'Data Visualization'],
 ['Pictorial map', 'Geology', 'Data Visualization'],
 ['Historical', 'Data Visualization'],
 ['Historical', 'Data Visualization'],
 ['Pictorial map', 'Data Visualization', 'Rivers'],
 ['Pictorial map', 'Data Visualization', 'Mountains; Rivers'],
 ['Historical', 'Data Visualization'],
 ['Historical', 'Data Visualization'],
 ['Celestial', 'Data Visualization'],
 ['Data Visualization', 'Ottoman Mapping'],
 ['Pictorial map', 'Religions', 'Data Visualization'],
 'Data Visualization',
 ['Energy', 'Data Visualization'],
 'Data Visualization',
 ['Data Visualization', 'Industry', 'Energy'],
 ['Historical', 'Data Visualization'],
 ['Historical'

In [93]:
def hashtags(subjects):
    if isinstance(subjects, list):
        result = []
        for s in subjects:
            if isinstance(s, list):
                group = ' '.join(f"#{str(i).lower().replace(' ', '')}" for i in s)
                result.append(group)
            else:
                result.append(f"#{str(s).lower().replace(' ', '')}")
        return result
    
    elif isinstance(subjects, str):
        return [f"#{subjects.lower().replace(' ', '')}"]
    
    else:
        return []

In [ ]:
keywords = hashtags(subjects)
keywords

['#datavisualization #art',
 '#datavisualization',
 '#historical #datavisualization',
 '#pictorialmap #historical #datavisualization',
 '#pictorialmap #datavisualization',
 '#pictorialmap #historical #datavisualization',
 '#religions #pictorialmap #celestial #datavisualization',
 '#pictorialmap #datavisualization',
 '#pictorialmap #geology #datavisualization',
 '#historical #datavisualization',
 '#historical #datavisualization',
 '#pictorialmap #datavisualization #rivers',
 '#pictorialmap #datavisualization #mountains;rivers',
 '#historical #datavisualization',
 '#historical #datavisualization',
 '#celestial #datavisualization',
 '#datavisualization #ottomanmapping',
 '#pictorialmap #religions #datavisualization',
 '#datavisualization',
 '#energy #datavisualization',
 '#datavisualization',
 '#datavisualization #industry #energy',
 '#historical #datavisualization',
 '#historical #datavisualization',
 '#datavisualization #political #statistical',
 '#historical #datavisualization',
 '#dat

In [102]:
len(keywords)

100

In [100]:
keywords[-1]

'#historical #datavisualization'

In [104]:
df = pd.DataFrame({
    'identifier': identifiers,
    'title': titles,
    'date': dates,
    'language': languages,
    'collection': collections,
    'url': urls,
    'keywords': keywords
})

df


,identifier,title,date,language,collection,url,keywords
0,dr_composite-map-of-tavola-1-60-map-of-the-wor...,(Composite map of) Tavola 1-60. (Map of the Wo...,1587,[None],david-rumsey-map-collection,https://archive.org/download/dr_composite-map-...,#datavisualization #art
1,dr_composite-map-mappemonde-dress-es-en-1450-p...,"(Composite map) Mappemonde dress es en 1450, p...","[1450, 1854]",[None],david-rumsey-map-collection,https://archive.org/download/dr_composite-map-...,#datavisualization
2,dr_page-4-adams-synchronological-chart-of-univ...,(Page 4) Adams' Synchronological Chart of Univ...,1881,[None],david-rumsey-map-collection,https://archive.org/download/dr_page-4-adams-s...,#historical #datavisualization
3,dr_composite-of-adams-synchronological-chart-o...,(Composite of) Adams' Synchronological Chart o...,1881,[None],david-rumsey-map-collection,https://archive.org/download/dr_composite-of-a...,#pictorialmap #historical #datavisualization
4,dr_the-histomap-of-evolution-earth-life-and-ma...,"The Histomap of Evolution: earth, life and man...",1942,[None],david-rumsey-map-collection,https://archive.org/download/dr_the-histomap-o...,#pictorialmap #datavisualization
...,...,...,...,...,...,...,...
95,dr_map-of-the-union-republics-autonomous-repub...,"Map of the Union Republics, Autonomous Republi...",1939,[None],david-rumsey-map-collection,https://archive.org/download/dr_map-of-the-uni...,#datavisualization
96,dr_a-new-map-of-the-world-there-will-be-a-cycl...,A new map of the world (There will be a cycle ...,1938,[None],david-rumsey-map-collection,https://archive.org/download/dr_a-new-map-of-t...,#datavisualization
97,dr_lipton-lunar-space-map-zaffo-11431000,Lipton Lunar Space Map. Zaffo.,1969,[None],david-rumsey-map-collection,https://archive.org/download/dr_lipton-lunar-s...,#datavisualization #space
98,dr_system-according-to-the-holy-scriptures-dra...,System According to the Holy Scriptures. Drawn...,1846,[None],david-rumsey-map-collection,https://archive.org/download/dr_system-accordi...,#celestial #pictorialmap #datavisualization #s...


In [107]:
df.to_csv(f"{download_dir}/{'ia-metadata-1.csv'}", index=False)
